In [2]:
import nltk

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
import spacy

In [6]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [7]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [8]:
import pickle

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
from nltk.stem import WordNetLemmatizer

In [55]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines =False)

In [73]:
data['publish_date'] = pd.to_datetime(data['publish_date'].astype(str), format = '%Y%m%d')

In [129]:
r_data = pd.DataFrame(columns=['publish_date','headline_text'])

In [130]:
k = data.iloc[0]['publish_date']
last = data.iloc[len(data)-1]['publish_date']
text =""
for i in range(len(data)):   
    if data.iloc[i]['publish_date'] == k:
        text = text + " " + data.iloc[i]['headline_text']
        if i == len(data)-1:
            temp = pd.DataFrame(data = [[k, text]], columns=['publish_date','headline_text'])
            r_data = r_data.append(temp)
#             r_data = r_data.reset_index(drop=True)
            text = ""
            k = k+pd.DateOffset(1) 
    else:
        temp = pd.DataFrame(data = [[k, text]], columns=['publish_date','headline_text'])
        r_data = r_data.append(temp)
#         r_data = r_data.reset_index(drop=True)
        text = ""
        k = k+pd.DateOffset(1)
r_data = r_data.reset_index(drop=True)

In [161]:
r_data.to_csv("abcnews.csv",mode='w')

In [10]:
data_new =  pd.read_csv('abcnews.csv', error_bad_lines =False,index_col=[0])

In [11]:
data_new.head(5)

,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting li...
1,2003-02-20,abattoir up for sale again academic upbeat ab...
2,2003-02-21,action wanted to lower indigenous unemploymen...
3,2003-02-22,act tourist attractions honoured at awards ni...
4,2003-02-23,act will struggle with public housing demand ...


In [12]:
data_new['publish_date'] = pd.DatetimeIndex(data_new['publish_date']).year

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data_new['headline_text']))

In [14]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [170]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words_nostops = pd.Series(data_words_nostops)
data_lemmatized = data_words_nostops.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

In [178]:
data_lemmatized = list(data_lemmatized)

In [188]:
with open('lemmatized_data.pickle', 'wb') as f:
    pickle.dump(data_lemmatized, f, pickle.HIGHEST_PROTOCOL)

In [16]:
with open('lemmatized_data.pickle', 'rb') as f:
    data_lemmatized = pickle.load(f)

In [17]:
temp = data_new

In [19]:
data_lemmatized = pd.Series(data_lemmatized)

In [20]:
temp['lemmatize'] = data_lemmatized

In [22]:
for i in range(15):
    globals()['trend{}'.format(i+2003)] = temp.loc[temp.publish_date == i+2003]

In [23]:
trend_list = [trend2003, trend2004, trend2005, trend2006, trend2007, trend2008, trend2009, trend2010, trend2011, trend2012, trend2013, trend2014, trend2015, trend2016, trend2017]

In [24]:
lemmatized = list(trend2015['lemmatize'])
id2word = corpora.Dictionary(lemmatized)
texts = lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [31]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.555267902073382

Coherence Score:  0.35103661919634926


In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.219097  0.002700       1        1  98.183533
2     -0.101555 -0.056847       2        1   1.788059
0     -0.117542  0.054148       3        1   0.028404, topic_info=      Category         Freq           Term        Total  loglift  logprob
term                                                                     
499    Default  1152.000000            win  1152.000000  30.0000  30.0000
120    Default   994.000000            day   994.000000  29.0000  29.0000
116    Default   795.000000            cup   795.000000  28.0000  28.0000
160    Default   445.000000          final   445.000000  27.0000  27.0000
503    Default  1017.000000          world  1017.000000  26.0000  26.0000
544    Default   353.000000           beat   353.000000  25.0000  25.0000
446    Default  1255.000000         sydney  1255.000000  24.0000  24.0000
256    Default   645.000000           live   645.000000  23.0000  23.0000
739    Default   360.000000      interview   360.000000  22.0000  22.0000
276    Default   891.000000      melbourne   891.000000  21.0000  21.0000
252    Default   526.000000         league   526.000000  20.0000  20.0000
59     Default   514.000000       brisbane   514.000000  19.0000  19.0000
3211   Default   407.000000          rugby   407.000000  18.0000  18.0000
771    Default   507.000000           lead   507.000000  17.0000  17.0000
204    Default   533.000000         hobart   533.000000  16.0000  16.0000
1064   Default   368.000000     grandstand   368.000000  15.0000  15.0000
128    Default   667.000000            die   667.000000  14.0000  14.0000
1313   Default   273.000000          point   273.000000  13.0000  13.0000
2766   Default   348.000000            nrl   348.000000  12.0000  12.0000
288    Default   669.000000           miss   669.000000  11.0000  11.0000
311    Default   747.000000           open   747.000000  10.0000  10.0000
2350   Default   199.000000          grand   199.000000   9.0000   9.0000
593    Default   383.000000           city   383.000000   8.0000   8.0000
121    Default   428.000000           dead   428.000000   7.0000   7.0000
57     Default   255.000000            box   255.000000   6.0000   6.0000
237    Default   854.000000           kill   854.000000   5.0000   5.0000
72     Default   659.000000            car   659.000000   4.0000   4.0000
478    Default   149.000000         update   149.000000   3.0000   3.0000
1590   Default   422.000000            afl   422.000000   2.0000   2.0000
32     Default  1665.000000      australia  1665.000000   1.0000   1.0000
...        ...          ...            ...          ...      ...      ...
18351   Topic3     0.011587    pollunation     0.576299   4.2596  -9.3533
18330   Topic3     0.011587         libido     0.576299   4.2596  -9.3533
18334   Topic3     0.011587       melonoma     0.576299   4.2596  -9.3533
18309   Topic3     0.011587         findon     0.576299   4.2596  -9.3533
18328   Topic3     0.011587      landlords     0.576299   4.2596  -9.3533
18319   Topic3     0.011587      heinrichs     0.576299   4.2596  -9.3533
18320   Topic3     0.011587        hellboy     0.576299   4.2596  -9.3533
18322   Topic3     0.011587          huang     0.576299   4.2596  -9.3533
18325   Topic3     0.011587  irrecoverable     0.576299   4.2596  -9.3533
18315   Topic3     0.011587       gilllard     0.576299   4.2596  -9.3533
18316   Topic3     0.011587        gravely     0.576299   4.2596  -9.3533
18314   Topic3     0.011587         gibney     0.576299   4.2596  -9.3533
18335   Topic3     0.011587         messis     0.576299   4.2596  -9.3533
18308   Topic3     0.011587         fiesta     0.576299   4.2596  -9.3533
18336   Topic3     0.011587    microtimber     0.576299   4.2596  -9.3533
18317   Topic3     0.011587      greenwald     0.576299   4.2596  -9.3533
18344   Topic3     0.011587            pec     0.576299   4.2

In [32]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.000*"man" + 0.000*"nsw" + 0.000*"country" + 0.000*"police" + 0.000*"new" '
  '+ 0.000*"call" + 0.000*"hour" + 0.000*"queensland" + 0.000*"say" + '
  '0.000*"australia"'),
 (1,
  '0.006*"lucrative" + 0.005*"condobolin" + 0.004*"flush" + 0.004*"daintree" + '
  '0.004*"wong" + 0.003*"rowers" + 0.003*"migratory" + 0.002*"observatory" + '
  '0.002*"medina" + 0.002*"query"'),
 (2,
  '0.000*"cup" + 0.000*"australia" + 0.000*"australian" + 0.000*"say" + '
  '0.000*"man" + 0.000*"nsw" + 0.000*"win" + 0.000*"police" + '
  '0.000*"queensland" + 0.000*"new"'),
 (3,
  '0.000*"say" + 0.000*"man" + 0.000*"police" + 0.000*"new" + '
  '0.000*"australia" + 0.000*"australian" + 0.000*"nsw" + 0.000*"country" + '
  '0.000*"win" + 0.000*"hour"'),
 (4,
  '0.000*"say" + 0.000*"police" + 0.000*"new" + 0.000*"win" + '
  '0.000*"australia" + 0.000*"government" + 0.000*"nsw" + 0.000*"man" + '
  '0.000*"wa" + 0.000*"rural"'),
 (5,
  '0.000*"nsw" + 0.000*"police" + 0.000*"say" + 0.000*"new" + '
  '0.000*"

In [184]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [189]:
with open('corpus_data.pickle', 'wb') as f:
    pickle.dump(corpus, f, pickle.HIGHEST_PROTOCOL)

In [190]:
corpus = pd.Series(corpus)
data_new['corpus'] = corpus

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=trend2003['corpus'],
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)